# Part I. ETL Pipeline for Pre-Processing the Files

#### Importing Python packages that we need in this project

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking the current working directory
print(os.getcwd())

# Getting the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Creatting a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-27-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# Printing the total number of rows 
##print(len(full_data_rows_list))
# Printing the list of event data rows
##print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding portion of your project. 

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# Creating conniction to the database that is installed locally
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # Creating a session to excute the quieries later on 
    session = cluster.connect()
    
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# Creating a keyspace named sparkify to hold our work in the database
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Connecting to the keyspace sparkify that's been created in the previous code
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries.

## The qustions that we need to create tables and SELECT statments to answer them

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Three tables are needded for each query

### The first table is song_in_session and it is for the first query

### The second table is users_songs and it is for the second query

### The third table is users_by_songs and it is for the third query

### Creating the first table "songs_in_session" for the first query

In [8]:
#Creating the first table named songs_in_session which containes the columns that are requierd in the first query along side with the sessionid and iteminsession to filter on them in the SELECT statment
query = "CREATE TABLE IF NOT EXISTS songs_in_session "
query = query + "(session_id int, iteminsession int, artist_name text, song_title text, song_length float, PRIMARY KEY (session_id, iteminsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
# Getting the csv file and looping till the end of the file to get all the requierd rows
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # Since we do not need the headers in the csv file , we use this code to skip the first line (headers)
    for line in csvreader:
# Creating an INSERT statment to insert the values into the first table songs_in_session
        query = "INSERT INTO songs_in_session (session_id, iteminsession, artist_name, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        #Converting some of the columns into the correct data type, since all the coulmns in the csv file are string
        session_id = int(line[8])
        iteminsession = int(line[3])
        song_length = float(line[5]) 
        # Assinging each column required in the query only to the values in INSERT statment
        session.execute(query, (session_id, iteminsession, line[0], line[9], song_length))

In [10]:
## Creating the SELECT statment for the first query and it should give us the artist name, song title and song length based on given session id and item in session
query = "select * from songs_in_session where session_id = 338 and iteminsession = 4"
try:
    #Run the SELECT statment
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    #Printing only the requierd columns 
    print (row.artist_name, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Creating the second table "users_songs" for the second query

In [11]:
#Creating the second table named users_songs which containes the columns that are requierd in the second query along side with the user_id and sessionid to filter on them in the SELECT statment and 
# iteminsession to order the results
query = "CREATE TABLE IF NOT EXISTS users_songs "
query = query + "(iteminsession int, user_id int, session_id int, artist_name text, song_title text, first_name text, last_name text, PRIMARY KEY (user_id, session_id, iteminsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)             

In [12]:
# Getting the csv file and looping till the end of the file to get all the requierd rows
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # Since we do not need the headers in the csv file , we use this code to skip the first line (headers)
    for line in csvreader:
# Creating an INSERT statment to insert the values into the second table users_songs
        query = "INSERT INTO users_songs (iteminsession, user_id, session_id, artist_name, song_title, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        #Converting some of the columns into the correct data type, since all the coulmns in the csv file are string
        session_id = int(line[8])
        iteminsession = int(line[3])
        user_id = int(line[10]) 
        # Assinging each column required in the query only to the values in INSERT statment
        session.execute(query, (iteminsession, user_id, session_id, line[0], line[9], line[1], line[4]))                    

In [13]:
## Creating the SELECT statment for the secnd query which will give us artist name, song title and user's first and last name based on given user id and session id
query = "select * from users_songs where user_id = 10 and session_id = 182"
try:
    #Run the SELECT statment
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    #Printing only the requierd columns 
    print (row.artist_name, row.song_title, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Creating the third table "users_by_song" for the third query

In [14]:
#Creating the third table named users_by_song which containes the columns that are requierd in the third query along side with the user_id and iteminseesion and song_title to filter on them in the SELECT statment
# and make the result unique
query = "CREATE TABLE IF NOT EXISTS users_by_song "
query = query + "(iteminsession int, user_id int, song_title text, first_name text, last_name text, PRIMARY KEY (song_title, user_id, iteminsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)      

In [15]:
# Getting the csv file and looping till the end of the file to get all the requierd rows
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # Since we do not need the headers in the csv file , we use this code to skip the first line (headers)
    for line in csvreader:
# Creating an INSERT statment to insert the values into the third table users_by_song
        query = "INSERT INTO users_by_song (iteminsession, user_id, song_title, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        #Converting some of the columns into the correct data type, since all the coulmns in the csv file are string
        iteminsession = int(line[3])
        user_id = int(line[10]) 
        # Assinging each column required in the query only to the values in INSERT statment
        session.execute(query, (iteminsession, user_id, line[9], line[1], line[4]))          

In [16]:
## Creating the SELECT statment for the third query which should give us the user's first and last name based on song title
query = "select * from users_by_song where song_title = 'All Hands Against His Own'"
try:
    #Run the SELECT statment
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    #Printing only the requierd columns 
    print (row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Dropping all three tables after we finished and before closing the session

In [17]:
# Dropping first table songs_in_session
query = "drop table songs_in_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# Dropping second table users_songs
query = "drop table users_songs"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# Dropping third table users_by_song
query = "drop table users_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Shutdown the session and the cluster after finishing the project

In [18]:
# Closing the session
session.shutdown()
# Closing the cluster
cluster.shutdown()